In [10]:
import pandas as pd
from tqdm import tqdm
import re

In [3]:
# Join the files
df = pd.DataFrame()
import glob


for filename in glob.glob('/Users/andrewzhuang/Desktop/Badminton Data/jnotebooks/2018 Tournaments/*.csv'):
    df_temp = pd.read_csv(filename)
    del df_temp['Unnamed: 0']
    df = df.append(df_temp)

for filename in glob.glob('/Users/andrewzhuang/Desktop/Badminton Data/jnotebooks/2017 Tournaments/*.csv'):
    df_temp = pd.read_csv(filename)
    del df_temp['Unnamed: 0']
    df = df.append(df_temp)
                                                                                                                                                                                         

In [6]:
def filter_name(string):
    del_chars = ['[',']','1','2','3','4','5','6','7','8','9','0']
    return_string = string
    for char in string:
        if char in del_chars:
            return_string = return_string.replace(char,'')
    
    return_string = return_string.strip()
    return (return_string)

In [7]:
#Take in line, player team you want to extract and singles or doubles
def split_players(x,player_num,s_or_d):
    #Need to filter out using regex
    return_val = re.findall("'(.*?)'",x)
    
    #Singles
    if s_or_d == 's':
        #only if there are two opponents
        if len(return_val) == 2 and player_num <= 1:
            for i in range(len(return_val)):
                return_val[i] = filter_name(return_val[i])
            return (return_val[player_num])
        else:
            return ('no opponent')
    
    #Doubles
    if s_or_d == 'd':
        #only if position exists
        if len(return_val) == 4 and player_num <= 1:
            for i in range(len(return_val)):
                return_val[i] = filter_name(return_val[i])
            #sort names alphbetically
            ret_list = sorted([return_val[player_num*2],return_val[player_num*2 + 1]])
            ret_str = filter_name(ret_list[0]) + ',' + ret_list[1]
            return (ret_str)
        else:
            return ('no opponent')

In [75]:
#This function will create a dictionary with trueskill rankings for each player
def create_event_dict_PM_and_TP(df):
    temp_dict = {}
    temp_set = set()
    for player in df['WINNER'].values:
        temp_set.add(player)
    for player in df['LOSER'].values:
        temp_set.add(player)
    
    for player in temp_set:
        temp_dict[player] = 0
    return (temp_dict)

def create_event_dict_WL(df):
    temp_dict = {}
    temp_set = set()
    for player in df['WINNER'].values:
        temp_set.add(player)
    for player in df['LOSER'].values:
        temp_set.add(player)
    
    for player in temp_set:
        temp_dict[player] = [0,0]
    return (temp_dict)

In [74]:
def update_win_loss(df,event_dict):
    for person in df['WINNER']:
        event_dict[person][0] += 1
    for person in df['LOSER']:
        event_dict[person][1] += 1
    return (event_dict)

def update_total_points(df,event_dict):
    for i in range(len(df)):
        event_dict[df.iloc[i]['WINNER']] += df.iloc[i]['WINNER POINTS']
        event_dict[df.iloc[i]['LOSER']] += df.iloc[i]['LOSER POINTS']
    return (event_dict)

def update_total_PLUSMINUS(df,event_dict):
    for i in range(len(df)):
        event_dict[df.iloc[i]['WINNER']] += df.iloc[i]['WINNER +-']
        event_dict[df.iloc[i]['LOSER']] += df.iloc[i]['LOSER +-']
    return (event_dict)

In [65]:
#stat num:
#         1 - winner total points
#         2 - loser total points
#         3 - winner set plus minus
#         4 - loser set plus minus

def player_point_split(x,stat_num):
    del_chars = ['[',']',"'",' ']
    return_string = x
    for char in x:
        if char in del_chars:
            return_string = return_string.replace(char,'')
    
    sets = return_string.split(',')
    
    winner_total_points = 0
    loser_total_points = 0
    
    winner_set_pm = 0
    loser_set_pm = 0

    for game in sets:
        if len(game) > 0:
            scores = game.split('-')
            winner_set_pm += int(scores[0]) - int(scores[1])
            loser_set_pm += int(scores[1]) - int(scores[0])
            winner_total_points += int(scores[0])
            loser_total_points += int(scores[1])
            
    if stat_num == 1:
        return (winner_total_points)
    elif stat_num == 2:
        return (loser_total_points)
    elif stat_num == 3:
        return (winner_set_pm)
    elif stat_num == 4:
        return (loser_set_pm)

df['WINNER POINTS'] = df['SCORES'].apply(lambda x: player_point_split(x,1))
df['LOSER POINTS'] = df['SCORES'].apply(lambda x: player_point_split(x,2))
df['WINNER +-'] = df['SCORES'].apply(lambda x: player_point_split(x,3))
df['LOSER +-'] = df['SCORES'].apply(lambda x: player_point_split(x,4))

In [79]:
MS_dict = create_event_dict_PM_and_TP(df_MS)



In [82]:
MS_dict = update_total_PLUSMINUS(df_MS,MS_dict)

In [85]:
players = []
rating = []
for player in MS_dict:
    players.append(player)
    rating.append(MS_dict[player])
df_results = pd.DataFrame()
df_results['players'] = players
df_results['+-'] = rating
df_results.sort_values(by = '+-', ascending = False)

players    +-
737                  Kevin Cordon  2361
1066                  Ygor Coelho  2284
699             Rosario Maddaloni  2084
922             Khosit Phetpradab  2071
313               Osleni Guerrero  1965
962                Brice Leverdez  1890
590               Misha Zilberman  1858
909         Suppanyu Avihingsanon  1854
862           Panji Ahmad Maulana  1820
884         Jason Anthony Ho-Shue  1815
1164        Kantaphon Wangcharoen  1792
1166                 Mark Caljouw  1791
588            Pannawit Thongnuam  1775
1054                  Lakshya Sen  1760
1175             Jonatan Christie  1753
697              Ng Ka Long Angus  1713
1328          Firman Abdul Kholik  1706
305       Shesar Hiren Rhustavito  1701
578                  Enzi Shafira  1679
768                   Hsu Jen Hao  1665
555           Krishna Adi Nugraha  1624
1334               Chou Tien Chen  1620
508                 Prannoy H. S.  1618
591            Luis Ramon Garrido  1597
213                Tommy Sugiarto  1590
1139        Ihsan Maulana Mustofa  1584
1308     Anthony Sinisuka Ginting  1560
686              Tien Minh Nguyen  1543
465                June Wei Cheam  1520
971           Georges Julien Paul  1504
...                           ...   ...
1248                  Tariq Sweet   -60
942            Yessine Vydelingum   -60
622             Francis Joey Wong   -60
387                 Manula Peires   -63
1290                  Tolo Kareng   -63
955        Arun Kumar Ashok Kumar   -63
310               Jatoonah Ashley   -66
257                 Isaac Senfuka   -69
446           Jaspar Woon Chai Yu   -69
294   Angel Argenis Marinez Ulloa   -72
144                  Heinoa Deane   -75
1340               Mathew Fogarty   -76
1387            Maxie Andrew Wong   -78
785                Youssef Hafouz   -78
104         Hussein Zayan Shaheed   -80
325             Andrey Parakhodin   -81
1173                 Chang Ho Kim   -84
214         Raul Blondet Martinez   -87
1353                 Teiva Politi   -93
1205        Michael Spencer-Smith  -104
648               Gustavo Morales  -105
1389      Yehezkiel Fritz Mainaky  -105
314                Chee Foong Lim  -105
6       Manuel Del Rosario Pargas  -111
131             Orkhan Galandarov  -112
730             Ousmane Ouedraogo  -117
281            Nicholas Bonkowsky  -126
1065   Pedro Arturo Garcia-Flores  -127
967                 Yan Tuck Chan  -128
948   Nathaniel Ernestan Sulistyo  -166

[1420 rows x 2 columns]

In [78]:
#Create dictionary and event dataframes sorted by date
MS_dict, WS_dict, MD_dict, WD_dict, XD_dict = {},{},{},{},{}

#Create the dataframes for each event
df_MS = df[df['EVENT'] == 'MS'].sort_values(by = 'match_week')
df_WS = df[df['EVENT'] == 'WS'].sort_values(by = 'match_week')
df_MD = df[df['EVENT'] == 'MD'].sort_values(by = 'match_week')
df_WD = df[df['EVENT'] == 'WD'].sort_values(by = 'match_week')
df_XD = df[df['EVENT'] == 'XD'].sort_values(by = 'match_week')

#Split players into winners and losers of a match
df_MS['WINNER'] = df_MS['PLAYERS'].apply(lambda x: split_players(x,0,'s'))
df_MS['LOSER'] = df_MS['PLAYERS'].apply(lambda x: split_players(x,1,'s'))
df_WS['WINNER'] = df_WS['PLAYERS'].apply(lambda x: split_players(x,0,'s'))
df_WS['LOSER'] = df_WS['PLAYERS'].apply(lambda x: split_players(x,1,'s'))
df_MD['WINNER'] = df_MD['PLAYERS'].apply(lambda x: split_players(x,0,'d'))
df_MD['LOSER'] = df_MD['PLAYERS'].apply(lambda x: split_players(x,1,'d'))
df_WD['WINNER'] = df_WD['PLAYERS'].apply(lambda x: split_players(x,0,'d'))
df_WD['LOSER'] = df_WD['PLAYERS'].apply(lambda x: split_players(x,1,'d'))
df_XD['WINNER'] = df_XD['PLAYERS'].apply(lambda x: split_players(x,0,'d'))
df_XD['LOSER'] = df_XD['PLAYERS'].apply(lambda x: split_players(x,1,'d'))

In [21]:
#Initialize the Ranking dictionaries!
MS_dict = create_event_dict_WL(df_MS)
WS_dict = create_event_dict_WL(df_WS)
MD_dict = create_event_dict_WL(df_MD)
WD_dict = create_event_dict_WL(df_WD)
XD_dict = create_event_dict_WL(df_XD)



In [22]:
MS_dict = update_win_loss(df_MS,MS_dict)

In [4]:
# Collect points per player, take in win percentage

In [23]:
MS_dict

{'Shubham Prajapati': [1, 1],
 'Femi Bioku': [0, 1],
 'Joo Ven Soong': [12, 7],
 'Niels VEENSTRA': [1, 1],
 'Soo Teck Zhi': [0, 1],
 'Gilmar Jones': [0, 1],
 'Manuel Del Rosario Pargas': [0, 2],
 'Wilmer Daniel Brea Nuñez': [2, 4],
 'Zvonimir Durkinjak': [7, 7],
 'Danny Chen': [1, 1],
 'Wei Chi Liu': [0, 2],
 'Shabahang Asgari': [0, 1],
 'Ahmad Ali': [0, 2],
 'Adam Mubiru': [0, 1],
 'Bongani Von Bodenstein': [1, 2],
 'Adam Mendrek': [13, 11],
 'Thanh Long Truong': [0, 2],
 'Manuel Mejia': [0, 1],
 'Chun-Wei Chen': [5, 6],
 'Rezha Akbar Raja Husain': [0, 2],
 'Bao Duc Duong': [1, 1],
 'Mohamed Mahmoud Djitli': [1, 1],
 'Izak Batalha': [0, 1],
 'Koki Watanabe': [18, 10],
 'Adel Hamek': [2, 3],
 'Javier Macagno': [1, 1],
 'Victor Alves': [0, 1],
 'Idrisse Issofo Mossagy': [0, 1],
 'Alexander Butovetstkiy': [0, 2],
 'Yonathan Ramlie': [3, 0],
 'Enrique Iglesia Cordero': [0, 1],
 'Luke Pearce': [3, 3],
 'Jose Guevara /': [3, 0],
 'Ondřej Král': [9, 2],
 'Kasper Lehikoinen': [2, 3],
 'Hemant

In [55]:
for game in df['SCORES'].apply(lambda x: player_point_split(x))[0]:
    scores = game.split('-')
    print(int(scores[0]))
    print(int(scores[1]))

21
14
22
20


In [66]:
df['WINNER POINTS'] = df['SCORES'].apply(lambda x: player_point_split(x,1))
df['LOSER POINTS'] = df['SCORES'].apply(lambda x: player_point_split(x,2))
df['WINNER +-'] = df['SCORES'].apply(lambda x: player_point_split(x,3))
df['LOSER +-'] = df['SCORES'].apply(lambda x: player_point_split(x,4))

In [67]:
df.head()

PLAYERS  \
0                  ['Jacob Nilsson', 'Kai Schaefer']   
1            ['Sabrina Jaquet', 'Natalia Perminova']   
2  ['Rodion Alimov', 'Alina Davletova', 'Ben Lane...   
3  ['Marcus Ellis [3]', 'Lauren Smith', 'Robin Ta...   
4  ['Bastian Kersaudy', 'Lea Palermo', 'Jelle Maa...   

                                  COUNTRIES                       SCORES  \
0                      ['[SWE] ', '[GER] ']           ['21-14', '22-20']   
1                      ['[SUI] ', '[RUS] ']  ['21-17', '13-21', '21-17']   
2  ['[RUS] ', '[RUS] ', '[ENG] ', '[ENG] ']           ['21-19', '27-25']   
3  ['[ENG] ', '[ENG] ', '[NED] ', '[NED] ']  ['17-21', '21-11', '21-15']   
4  ['[FRA] ', '[FRA] ', '[NED] ', '[NED] ']           ['23-21', '21-12']   

  EVENT match_week  WINNER POINTS  LOSER POINTS  WINNER +-  LOSER +-  
0   MS    20180424             43            34          9        -9  
1   WS    20180424             55            55          0         0  
2   XD    20180424             48            44          4        -4  
3   XD    20180424             59            47         12       -12  
4   XD    20180424             44            33         11       -11